In [ ]:
import torch
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image  # Import Image for handling image files

# Step 3: Dataset Preparation
data_path = 'D:\\GBC SEM2\\Deep Learning\\Project\\Train'

# Apply additional preprocessing or augmentations
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

# Custom Dataset class
class TrafficSignDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.images = list(Path(data_path).rglob('*.png'))  # Adjusted to .png format

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        # Adjust to the .png format and label paths if necessary
        label_path = str(img_path).replace('.png', '.txt')  # Assuming labels are stored with .txt extension
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        with open(label_path, 'r') as f:
            labels = f.read().splitlines()
        
        return image, labels

# Adjusted paths to match the actual folder names
train_dataset = TrafficSignDataset(os.path.join(data_path, 'Train'), transform=transform)
val_dataset = TrafficSignDataset(os.path.join(data_path, 'Test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Step 4: Load YOLOv5 Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Step 5: Train the Model with Augmented Data and Hyperparameters
epochs = 50
batch_size = 16
img_size = 640
learning_rate = 0.01
momentum = 0.937

# YOLOv5 Training Loop
model.train(data=data_path, epochs=epochs, batch_size=batch_size, imgsz=img_size, lr0=learning_rate, momentum=momentum)

# Step 6: Evaluate the Model
results = model(val=True)
print(results)

# Visualization of training progress
plt.plot(results['metrics']['precision'], label='Precision')
plt.plot(results['metrics']['recall'], label='Recall')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Precision and Recall over Epochs')
plt.legend()
plt.show()

# Step 7: Save the Trained Model
model_path = 'yolov5_traffic_sign.pt'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

# Step 8: Inference on New Data
model.load_state_dict(torch.load(model_path))
model.eval()

img = 'path/to/test_image.png'  # Adjusted to .png format
results = model(img)
results.show()


In [ ]:
import torch
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Step 1: Load Images and Labels
data = []
labels = []
classes = 43  # Change this to 42 if your dataset has 42 classes
cur_path = os.getcwd()

# Retrieving the images and their labels
for i in range(classes):
    path = os.path.join(cur_path, 'train', str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(os.path.join(path, a)).convert("RGB")
            image = image.resize((30, 30))  # Resize to match the CNN preprocessing
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(f"Error loading image {a}: {e}")

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f"Train/Test shapes: {X_train.shape}, {X_test.shape}, {y_train.shape}, {y_test.shape}")

# Converting the labels into one-hot encoding (this step might be skipped for YOLOv5)
y_train = to_categorical(y_train, classes)
y_test = to_categorical(y_test, classes)

# Step 2: Create a Custom Dataset Class
class TrafficSignDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return image, label

# Step 3: Define Transforms and Load Data
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

train_dataset = TrafficSignDataset(X_train, y_train, transform=transform)
val_dataset = TrafficSignDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)



In [ ]:
# Load YOLOv5 Model
from models.yolo import Model

# Load model weights
model = Model(cfg='D:\GBC SEM2\Deep Learning\Project\yolov5\models\yolov5s.yaml')

model.load_state_dict(torch.load('D:\GBC SEM2\Deep Learning\Project\yolov5s.pt'))



In [28]:
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'requests>=2.32.0'] not found, attempting AutoUpdate...


Using cache found in C:\Users\Admin/.cache\torch\hub\ultralytics_yolov5_master


Retry 1/2 failed: Command 'pip install --no-cache-dir "gitpython>=3.1.30" "requests>=2.32.0" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "gitpython>=3.1.30" "requests>=2.32.0" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "gitpython>=3.1.30" "requests>=2.32.0" ' returned non-zero exit status 1.


YOLOv5  2024-8-14 Python-3.10.9 torch-2.3.0+cpu CPU

Fusing layers... 
YOLOv5s_v6 summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [34]:
# Step 5: Train the Model with Augmented Data and Hyperparameters
epochs = 50
batch_size = 16
img_size = 640
learning_rate = 0.01
momentum = 0.937

# YOLOv5 Training Loop
model.train(data=data, epochs=epochs, batch_size=batch_size, imgsz=img_size, lr0=learning_rate, momentum=momentum)

# Step 6: Evaluate the Model
results = model(val=True)
print(results)

# Visualization of training progress
plt.plot(results['metrics']['precision'], label='Precision')
plt.plot(results['metrics']['recall'], label='Recall')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Precision and Recall over Epochs')
plt.legend()
plt.show()

# Step 7: Save the Trained Model
model_path = 'yolov5_traffic_sign.pt'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

# Step 8: Inference on New Data
model.load_state_dict(torch.load(model_path))
model.eval()

img = 'path/to/test_image.png'  # Adjusted to .png format
results = model(img)
results.show()


TypeError: Module.train() got an unexpected keyword argument 'data'